<a href="https://colab.research.google.com/github/Alexander-T2/TextQuest_Sasha_Masha/blob/main/%D0%9C%D0%B0%D1%82%D0%B0%D0%BD_%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Учимся исследовать экстремумы квадратичных функций  

Данный блокнот демонстрирует **пошаговое исследование** функции  

$$
f(x,y)=A\,x^{2}+2B\,x\,y+C\,y^{2}+P\,x+Q\,y+R
$$

где коэффициенты вводит пользователь.  
После выполнения вы сможете  

1. находить стационарные точки через ∇f;  
2. классифицировать их с помощью матрицы Гессе;  
3. понять, что делать, когда определитель матрицы Гессе равен 0.  

> Все вычисления интерактивны: меняйте коэффициенты — сразу получаете новый анализ и график.


In [17]:
# Введите числа и нажмите Enter (пусто = значение по умолчанию)
def read_float(name, default):
    s = input(f"{name}  (default = {default}): ")
    return float(s) if s.strip() else default

print("Коэффициенты квадратичной функции")
print("f(x,y) = A·x² + 2B·x·y + C·y² + P·x + Q·y + R\n")

coeff = {
    'A': read_float('A', 1.0),
    'B': read_float('B', 0.0),
    'C': read_float('C', 1.5),
    'P': read_float('P', 0.0),
    'Q': read_float('Q', 0.0),
    'R': read_float('R', 0.0)
}

print("\nСохранённые коэффициенты:", coeff)


Коэффициенты квадратичной функции
f(x,y) = A·x² + 2B·x·y + C·y² + P·x + Q·y + R

A  (default = 1.0): 1
B  (default = 0.0): 2
C  (default = 1.5): 4
P  (default = 0.0): 0
Q  (default = 0.0): 0
R  (default = 0.0): 0

Сохранённые коэффициенты: {'A': 1.0, 'B': 2.0, 'C': 4.0, 'P': 0.0, 'Q': 0.0, 'R': 0.0}


In [18]:
# !pip -q install plotly            # первый раз может потребоваться установка
import numpy as np, sympy as sp, plotly.graph_objects as go, plotly.io as pio
pio.renderers.default = "colab"

x, y = sp.symbols('x y')
f_expr = (coeff['A']*x**2 + 2*coeff['B']*x*y + coeff['C']*y**2 +
          coeff['P']*x   + coeff['Q']*y   + coeff['R'])
f_num  = sp.lambdify((x, y), f_expr, 'numpy')

grid = np.linspace(-3, 3, 180)
X, Y = np.meshgrid(grid, grid)
Z    = f_num(X, Y)

fig = go.Figure(go.Surface(x=X, y=Y, z=Z,
                           colorscale='RdBu', showscale=False))
fig.update_layout(
    title=(f"f(x,y) при A={coeff['A']}, B={coeff['B']}, C={coeff['C']}, "
           f"P={coeff['P']}, Q={coeff['Q']}, R={coeff['R']}"),
    width=850, height=600,
    scene_camera=dict(eye=dict(x=1.25, y=1.25, z=0.9)),
    margin=dict(l=0, r=0, t=50, b=0)
)
fig.show()

print("График можно вращать и приближать)")


График можно вращать и приближать)


  
Цвет отражает значение функции: красное выше, синее ниже.  
Следующий шаг — поиск точек с нулевым градиентом.


##Градиент и стационарные точки — что, зачем и как  

**Что такое градиент?**  
Пусть  
$$
f(x,y)=A\,x^{2}+2B\,x\,y+C\,y^{2}+P\,x+Q\,y+R .
$$  

Его градиент — это вектор частных производных  

$$
\nabla f(x,y)=\bigl(f_x(x,y),\,f_y(x,y)\bigr).
$$  

Он указывает направление самого быстрого роста функции.  



 **Почему именно ∇f = 0?**  
* Если  ∇f = 0 можно сдвинуться в сторону уменьшения или увеличения \(f\).  
* Поэтому **локальный экстремум возможен только там, где градиент обнуляется**.  
  Эти точки называют *стационарными*.  



**Как мы их ищем?**  
Для квадратичной функции \(f_x\) и \(f_y\) — линейные выражения,  
поэтому система  

$$
\begin{cases}
f_x(x,y)=0,\\
f_y(x,y)=0
\end{cases}
$$  

линейна.  
Команда `sympy.solve` находит все её решения-пары \((x^\*,y^\*)\) за доли секунды.  



> Итог: получаем **множество стационарных точек** — кандидатов в минимум, максимум или седло.  
> Кто есть кто, решит следующий шаг с матрицей Гессе.


In [21]:
# градиент и решение системы ∇f = 0
grad_fx = [sp.diff(f_expr, v) for v in (x, y)]
stationary = sp.solve(grad_fx, (x, y), dict=True)

if stationary:
    print("Стационарные решения:")
    for sol in stationary:
        if sol[x].free_symbols or sol[y].free_symbols:
            print("  семейство:", sol)
        else:
            print(f"  точка: ({sol[x]}, {sol[y]})")
else:
    print("Стационарных точек нет.")


Стационарные решения:
  семейство: {x: -2.0*y}


##Матрица Гессе: классификация стационарных точек

### Основные понятия

**Матрица Гессе** для функции двух переменных - симметричная матрица вторых производных:

$$
H(x,y) =
\begin{pmatrix}
f_{xx}(x,y) & f_{xy}(x,y) \\
f_{yx}(x,y) & f_{yy}(x,y)
\end{pmatrix}
$$

Эта матрица характеризует локальную кривизну поверхности в окрестности точки.

### Критерий второго порядка

**1. Локальный минимум**  
  ∘ det H > 0 и fₓₓ > 0

  ∘ Матрица положительно определена  
  ∘ Все собственные значения > 0  

**2. Локальный максимум**  
  ∘ det H > 0 и fₓₓ < 0  

  ∘ Матрица отрицательно определена  
  ∘ Все собственные значения < 0  

**3. Седловая точка**  
  ∘ det H < 0

  ∘ Собственные значения разных знаков  
  ∘ Поверхность имеет гиперболическую форму  

**4. Вырожденный случай (det H = 0 )**  
  ∘ Минимум одно нулевое собственное значение  
  ∘ В одном направлении поведение плоское  
  ∘ Требуется дополнительный анализ:  
    • Разложение до высших производных  
    • Численное исследование приращений  


*Примечание: tr H = fₓₓ + fᵧᵧ можно использовать вместо fₓₓ*


In [24]:
# вычисляем Гессе
H = sp.hessian(f_expr, (x, y))

def classify(pt):
    H_pt = np.array(H.subs(pt)).astype(float)
    det  = np.linalg.det(H_pt)
    tr   = np.trace(H_pt)

    if abs(det) > 1e-8:
        if det > 0 and tr > 0: return "локальный минимум"
        if det > 0 and tr < 0: return "локальный максимум"
        return "седловая точка"
    else:
        eig = np.linalg.eigvals(H_pt)
        pos, neg = (eig > 1e-9).sum(), (eig < -1e-9).sum()
        if pos and not neg: return "минимум (плоская долина)"
        if neg and not pos: return "максимум (плоское плато)"
        return "седло / плоскость (λ = 0)"

#--- запускаем классификацию ТОЛЬКО если найдётся хоть одна точка с det≠0
run_classification = False
if stationary:
    for s in stationary:
        det_val = np.linalg.det(np.array(H.subs(s)).astype(float))
        if abs(det_val) > 1e-8:
            run_classification = True
            break

if run_classification:
    print("Классификация стационарных точек:")
    for s in stationary:
        print(f"{s} → {classify(s)}")
else:
    print("det H ≈ 0 для всех стационарных точек – используйте проверку по определению.")


det H ≈ 0 для всех стационарных точек – используйте проверку по определению.


##Численная проверка по определению (при det H ≈ 0)

Когда матрица Гессе вырождена (det H ≈ 0), стандартный критерий не работает.
В этом случае анализируем экстремум **напрямую по определению**:

### Методика проверки:

1. **Формируем окрестность**:
   - Радиус δ = 10⁻³ (регулируемый параметр)
   - Углы θ ∈ [0, 2π) с разумным шагом дискретизации

2. **Вычисляем приращения**:
   Для точек на окружности:
   \[
   (x, y) = (x^* + δ\cosθ, y^* + δ\sinθ)
   \]
   Считаем разность значений:
   \[
   Δf_θ = f(x,y) - f(x^*,y^*)
   \]

3. **Интерпретация результатов**:
   - Если все Δf_θ > 0 → строгий локальный минимум
   - Если все Δf_θ < 0 → строгий локальный максимум
   - Если есть как положительные, так и отрицательные Δf_θ → седловая точка
   - Если все |Δf_θ| ≤ ε (машинный ноль) → плоское плато

### Реализация в коде:

1. Для каждой стационарной точки вычисляем det H
2. Проверяем условие вырожденности:
   - Если |det H| < 10⁻⁸ → активируем численную проверку
   - Иначе → применяем стандартный критерий 2-го порядка
3. Параметры δ (радиус) и ε (точность) настраиваются

*Примечание: для сложных случаев можно увеличивать радиус δ или уменьшать шаг по θ*
"""

In [28]:
# ------------------------------------------------------------
# Числовая проверка только при det H ≈ 0
# ------------------------------------------------------------
def classify_by_definition(x0, y0, delta=1e-3, n_dirs=360, tol=1e-9):
    thetas = np.linspace(0, 2*np.pi, n_dirs, endpoint=False)
    pos = neg = False
    f0  = f_num(x0, y0)
    for th in thetas:
        dx, dy = delta*np.cos(th), delta*np.sin(th)
        df = f_num(x0+dx, y0+dy) - f0
        if   df >  tol: pos = True
        elif df < -tol: neg = True
        if pos and neg:
            return "седло (по определению)"
    if pos and not neg: return "минимум (по определению)"
    if neg and not pos: return "максимум (по определению)"
    return "плоская область / неопред."

# --- собираем решения с det H ≈ 0 ---------------------------------------
degenerate_pts = []
if stationary:
    for sol in stationary:
        det_val = np.linalg.det(np.array(H.subs(sol)).astype(float))
        if abs(det_val) < 1e-8:
            degenerate_pts.append(sol)

# --- вывод ---------------------------------------------------------------
if degenerate_pts:
    print("Числовая проверка для точек с det H ≈ 0:")
    for sol in degenerate_pts:

        # выражения координат (если одной переменной нет в dict — берём саму переменную)
        expr_x = sol.get(x, x)
        expr_y = sol.get(y, y)

        # есть ли свободные символы (параметры) ?
        free_syms = expr_x.free_symbols.union(expr_y.free_symbols)
        if free_syms:                      # семейство
            t = list(free_syms)[0]         # берём любой параметр
            rep = {t: 0}                   # подставляем 0
            x0 = float(expr_x.subs(rep))
            y0 = float(expr_y.subs(rep))
            res = classify_by_definition(x0, y0)
            print(f"  семейство {sol}  →  {res} (проверка при {t}=0)")
        else:                              # обычная числовая точка
            x0, y0 = float(expr_x), float(expr_y)
            res = classify_by_definition(x0, y0)
            print(f"  ({x0:.4f}, {y0:.4f}) → {res}")
else:
    print("Все стационарные точки имеют det H ≠ 0; "
          "критерий второго порядка уже дал окончательный ответ.")


Числовая проверка для точек с det H ≈ 0:
  семейство {x: -2.0*y}  →  минимум (по определению) (проверка при y=0)


##Визуализация стационарных решений на 3-D поверхности  

* **Красный крест** — изолированная стационарная точка  
  * минимум лежит на «дне чаши»;  
  * максимум — на «вершине холма»;  
  * седло — на переходе с хребта в долину.  

* **Чёрная пунктирная линия** — семейство стационарных точек, возникающее при  
  \(\det H = 0\). Вдоль этой линии функция не изменяется — «плоская» долина, плато или хребет.  
  Проверка типа такого семейства делается по определению (см. шаг 6).  


In [29]:
# ------------------------------------------------------------
# Отображаем стационарные решения на 3-D поверхности.
# • если det H ≠ 0  →   одиночная точка (красный крест);
# • если det H ≈ 0  →   семейство: рисуем линию (пунктир) + подпись.
# ------------------------------------------------------------

tol = 1e-8                     # порог «det ≈ 0»
t_range = np.linspace(-3, 3, 150)   # параметры для линии-семейства

for sol in stationary:
    det_val = np.linalg.det(np.array(H.subs(sol)).astype(float))

    expr_x = sol.get(x, x)
    expr_y = sol.get(y, y)
    free   = list(expr_x.free_symbols | expr_y.free_symbols)

    # -------- обычная изолированная точка ---------------------------------
    if abs(det_val) > tol and not free:
        x0 = float(expr_x)
        y0 = float(expr_y)
        z0 = f_num(x0, y0)

        fig.add_trace(go.Scatter3d(
            x=[x0], y=[y0], z=[z0],
            mode='markers',
            marker=dict(symbol='cross', size=12, color='red',
                        line=dict(width=3)),
            name="изолир. точка"))

    # -------- семейство (det ≈ 0) -----------------------------------------
    else:
        # берём первый свободный параметр  → t
        t = free[0] if free else sp.symbols('t')

        x_t = sp.lambdify(t, expr_x, 'numpy')
        y_t = sp.lambdify(t, expr_y, 'numpy')

        xs = x_t(t_range)
        ys = y_t(t_range)
        zs = f_num(xs, ys)

        fig.add_trace(go.Scatter3d(
            x=xs, y=ys, z=zs,
            mode='lines',
            line=dict(color='black', dash='dot'),
            name=f"семейство: {sp.pretty(expr_x)} , {sp.pretty(expr_y)}"))

fig.show()


##Случай с det H = 0 - альтернативный вариант анализа:

Когда определитель матрицы Гессе равен нулю, это означает, что матрица второго порядка **вырождена**, и классический критерий экстремума не даёт однозначного ответа.

### Общая стратегия исследования:

1. **Вычисляем приращение функции**:

   $$
   \Delta f \;=\;
   f\bigl(x+\Delta x,\;y+\Delta y\bigr)
   \;-\;
   f\bigl(x,y\bigr).
   $$

2. **Разкладываем Δf в ряд Тейлора** и находим первый ненулевой член (порядка k ≥ 3, так как члены 1-го и 2-го порядков равны нулю):

   - Если этот член **> 0** для всех направлений → точка **минимума**
   - Если **< 0** для всех направлений → точка **максимума**
   - Если знак зависит от направления → **седловая точка**

### Особенность квадратичных функций

Для чисто квадратичной формы нулевое собственное значение λ означает, что вдоль соответствующего направления функция ведёт себя как:

- линейная функция
- или даже константа

В этом случае экстремума не существует — получается плоская долина, плато или хребет.

---

**Важное замечание**:  
При det H = 0 дальнейший анализ **сильно зависит от вида функции**:

- Для полиномов высших степеней (кубических и выше) необходимо учитывать производные третьего и более высоких порядков
- Для функций с экспонентами, тригонометрией и другими сложными элементами может потребоваться специальное разложение или численный анализ приращения функции